# Imports / Installs

In [ ]:
pip install transformers xgboost datasets huggingface_hub ankh

  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/98/46/f6a79f944d5c7763a9bc13b2aa6ac72daf43a6551f5fb03bccf0a9c2fec1/transformers-4.33.3-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/119.9 kB ? eta -:--:--
     --- ------------------------------------ 10.2/119.9 kB ? eta -:--:--
     ----------------------------------- -- 112.6/119.9 kB 1.3 MB/s eta 0:00:01
     -------------------------------------- 119.9/119.9 kB 1.2 MB/s eta 0:00:00
  Obtaining dependency information for ankh from https://files.pythonhosted.org/packages/74/a7/ea313596f57c14890f1985ae7a7554698571c239108fa799b352a34441b7/ankh-1.10.0-py3-none-any.whl.metadata
  Obtaining dependency information for datasets from https://files.pythonhosted.org/packages/09/7e/fd4d6441a541dba61d0acb3c1fd5df53214c2e9033854e837a99dd9e0793/datasets-2.14.5-py3-none-any.whl.metadata
  Obtaining dependency information for pandas from https://files.pythonhosted.org/packa

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as f
import ast
import pandas as pd
import xgboost as xgb
import numpy as np
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Lasso
from sklearn.feature_selection import RFE
from sklearn.metrics import mutual_info_score
from sklearn.preprocessing import StandardScaler
from tqdm.auto import tqdm
from transformers import AutoTokenizer, AutoModel
from datasets import Dataset, load_dataset

# Extract embeddings

In [3]:
dataset = load_dataset('lhallee/ec_feature_ranking', split='train')
dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/530876 [00:00<?, ? examples/s]

Dataset({
    features: ['Entry', 'EC number', 'Sequence', '1st', 'Class', 'group'],
    num_rows: 530876
})

In [ ]:
model_paths = ['facebook/esm2_t6_8M_UR50D', 'facebook/esm2_t12_35M_UR50D', 'Rostlab/prot_bert',
                'facebook/esm2_t30_150M_UR50D', 'facebook/esm2_t33_650M_UR50D',
                'facebook/esm2_t36_3B_UR50D', 'facebook/esm2_t48_15B_UR50D']

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # Check if GPU is available

for model_path in model_paths:
    # Load the model and tokenizer
    model = AutoModel.from_pretrained(model_path).to(device)  # Send the model to the GPU
    tokenizer = AutoTokenizer.from_pretrained(model_path)

    # Initialize an empty list to store the embeddings
    embeddings = []

    # Iterate over the sequences
    for seq in tqdm(seqs, desc=model_path):
        # Tokenize the sequence and get the model's output
        inputs = tokenizer(seq, return_tensors='pt').input_ids.to(device)  # Send the inputs to the GPU
        outputs = model(inputs)
        # Get the last hidden state and compute the mean
        embedding = outputs.last_hidden_state.mean(dim=0).cpu().detach().numpy()
        # Append the mean embedding to the list
        embeddings.append(embedding)

    # Add the embeddings as a new column in the dataset
    dataset['train'][model_path] = embeddings
    dataset.push_to_hub('lhallee/ec_feature_ranking')

    # Reset GPU memory
    del model
    torch.cuda.empty_cache()

# Feature ranking

In [ ]:
def hyper_param_search(df, targets, cv, model_params, num_runs):
    X = np.array(df)
    total_scores = pd.DataFrame()  # empty DataFrame to store scores from all iterations

    for target_col in tqdm(targets.columns):
        y = np.array(targets[target_col])
        scores = []  # Score list for cv
        # Randomized search CV on all models above
        for model_name, mp in model_params.items():
            clf = RandomizedSearchCV(mp['model'], mp['params'], n_iter=num_runs, cv=cv, return_train_score=False, random_state=42)
            clf.fit(X, y)
            scores.append({
                'model': model_name,
                'best_score': clf.best_score_,
                'best_params': clf.best_params_,
                'target_col': target_col  # adding target column name as an additional feature
            })

        cv_df = pd.DataFrame(scores, columns=['model','best_score','best_params', 'target_col'])  # create dataframe with each iteration's scores
        total_scores = pd.concat([total_scores, cv_df])  # concatenate the new scores with the total scores

    return total_scores.reset_index(drop=True)  # reset the index for the final DataFrame


def lasso_ranking(X, y, step_a):
    # Initialize
    alpha = 0
    last_coef = np.ones(len(X.columns))
    ranking = pd.DataFrame(columns=['Lasso order', 'Coef'])
    dropped_features = set()  # This set will store the features that have already been dropped

    # Scale features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # While loop until all coefficients are 0
    while np.any(last_coef != 0):
        las = Lasso(alpha)
        las.fit(X_scaled, y)
        coef = las.coef_

        # Check which coefficients have just become 0
        just_zeroed = (last_coef != 0) & (coef == 0)
        zeroed_features = list(X.columns[just_zeroed])

        # Update ranking
        for feature in zeroed_features:
            # Only add feature to ranking if it hasn't already been dropped
            if feature not in dropped_features:
                ranking = ranking.append({'Lasso order': feature, 'Coef': alpha,}, ignore_index=True)
                dropped_features.add(feature)

        # Update last_coef
        last_coef = coef

        # Increase alpha
        alpha += step_a

    # Sort ranking in descending order of alpha (ascending order of importance)
    ranking = ranking.sort_values(by='Coef', ascending=False).reset_index(drop=True)

    return ranking



def ranking(df, targets, hypers_rf, hypers_xb, alpha):
    X = np.array(df)
    total_ranking = pd.DataFrame()

    for target_col in targets.columns:
        y = np.array(targets[target_col])
        rf_hyper = hypers_rf[hypers_rf['target_col'] ==
                             target_col].reset_index(drop=True)['best_params'].apply(ast.literal_eval)[0]
        xb_hyper = hypers_xb[hypers_xb['target_col'] ==
                             target_col].reset_index(drop=True)['best_params'].apply(ast.literal_eval)[0]
        lasso_ranks = lasso_ranking(df, targets[target_col], alpha)
        # Feature importance from RandomForest
        rf = RandomForestClassifier(**rf_hyper)
        rf.fit(X, y)
        rf_importances = pd.DataFrame({'RF order':df.columns, 'Coef':rf.feature_importances_}).sort_values(by='Coef', ascending=False).reset_index(drop=True)

        # Feature importance from XGBoost
        xg = xgb.XGBClassifier(use_label_encoder=False, eval_metric='error', **xb_hyper)
        xg.fit(X, y)
        xg_importances = pd.DataFrame({'XGB order':df.columns, 'Coef':xg.feature_importances_}).sort_values(by='Coef', ascending=False).reset_index(drop=True)

        # Feature selection from Recursive Feature Elimination
        estimator = xgb.XGBClassifier(use_label_encoder=False, eval_metric='error', **xb_hyper)
        selector = RFE(estimator, n_features_to_select=X.shape[1], step=1)
        selector = selector.fit(X, y)
        rfe_support = selector.get_support()
        rfe_importances = pd.DataFrame({'RFE order': df.columns, 'Coef': range(len(df.columns))})
        rfe_importances['Coef'] = np.where(rfe_importances['RFE order'].isin(df.columns[rfe_support]),
                                                     rfe_importances['Coef'], np.nan)

        # Mutual Information
        mi_scores = []
        for feature in df.columns:
            mi = mutual_info_score(df[feature], y)
            mi_scores.append(mi)
        mi_df = pd.DataFrame({'MI order': df.columns, 'Coef': mi_scores}).sort_values(by='Coef', ascending=False).reset_index(drop=True)

        # Create a dataframe for the current target column, add 'target_col' column
        ranks = pd.concat([lasso_ranks, rf_importances, xg_importances, rfe_importances, mi_df], axis=1)
        ranks['target_col'] = target_col

        total_ranking = pd.concat([total_ranking, ranks])
        


    total_ranking.reset_index(drop=True, inplace=True)
    return total_ranking


def voting(df):
    unique_targets = df['target_col'].unique()
    all_scores = {}
    for target in unique_targets:
        final_scores = {}
        rows = df[df['target_col'] == target]
        lasso = rows['Lasso order'].tolist()
        rf = rows['RF order'].tolist()
        xgb = rows['XGB order'].tolist()
        rfe = rows['RFE order'].tolist()
        mi = rows['MI order'].tolist()
        w1, w2, w3, w4, w5 = 0.3, 0.2, 0.2, 0.1, 0.1
        # Lists and weights combined for convenience
        lists_and_weights = [(lasso, w1), (rf, w2), (xgb, w3), (rfe, w4), (mi, w5)]
        # Calculate final scores
        for feature_list, weight in lists_and_weights:
            for i, feature in enumerate(reversed(feature_list)):
                if feature not in final_scores:
                    final_scores[feature] = 0
                final_scores[feature] += (i+1) * weight

        # Sort features by final scores in descending order
        final_ranking = sorted(final_scores.items(), key=lambda item: item[1], reverse=True)
        all_scores[target] = final_ranking
        
    return all_scores


def get_data(path, columns):
    df = pd.read_csv(path)
    labels = ['Class', '1st']
    y = df[labels]
    X = df[columns]
    return X, y

In [ ]:
# hyperparams
max_depth = [int(x) for x in np.linspace(1, 1000, num = 10)] #Max depth list for trees
max_depth.append(None)

model_params = {
    'XGBoost':{
        'model':XGBClassifier(use_label_encoder=False, eval_metric='error'),
        'params' : {
            'max_depth': [int(x) for x in np.linspace(1, 50, num = 10)],
            'alpha': [int(x) for x in np.linspace(0.1, 10, num = 100)],
            'learning_rate': [int(x) for x in np.linspace(0.1, 5, num = 10)],
            'n_estimators':[int(x) for x in np.linspace(start = 5, stop = 1000, num = 10)]
        }
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [int(x) for x in np.linspace(start = 5, stop = 1000, num = 10)],
            'max_features': ['log2'],
            'max_depth': max_depth
        }
    }
}

In [ ]:
X, y = get_data(data_dict[ethnic])
hypers = hyper_param_search(X, y, 5, model_params, 50)
xb_scores = hypers[hypers['model'] == 'XGBoost'].reset_index(drop=True)
rf_scores = hypers[hypers['model'] == 'random_forest'].reset_index(drop=True)
ranks = ranking(X, y, rf_scores, xb_scores)
scoring = voting(ranks)

In [ ]:
for key in list(scoring.keys()):
    # Split the features and scores
    features, scores = zip(*scoring[key])
    # Limit to top 100 features
    features = features[:100]
    scores = scores[:100]
    # Create a figure and a set of subplots
    fig, ax = plt.subplots(figsize=(10, 15))  # Increase the size of your figure
    fig.patch.set_facecolor('white')

    ax.barh(features, scores, color='blue', alpha=0.6)
    ax.invert_yaxis()

    label_opts = {'color': 'black', 'bbox': dict(facecolor='white', edgecolor='none')}
    ax.set_xlabel('Score', **label_opts)
    ax.set_ylabel('Feature', **label_opts)
    ax.set_title(f'EC {key} ranking model {type}')

    ax.tick_params(axis='both', which='both', labelsize=8, labelcolor='black', colors='black')  # Reduce font size

    plt.yticks(rotation=45)  # Rotate y-axis labels

    plt.tight_layout()
    plt.savefig(f'ec_ranking_{key}_{type}.png', bbox_inches='tight', transparent=False, dpi=300)
    plt.show()

# Correlation analysis